In [1]:
"""
conda env list
conda activate base
conda list 
pip install aiohttp
"""

'\nconda env list\nconda activate base\nconda list \npip install aiohttp\n'

In [1]:
from google.cloud import bigquery
import pandas as pd

In [2]:
import asyncio
import time

import aiohttp
from aiohttp import ClientSession
# from typing import Any, Iterable, List, Tuple, Callable
# import os

# import requests

In [3]:
print(__name__)

__main__


In [4]:
# groovy nested fx

async def asyncio_request_batch(url_list):
    
    async def asyncio_request(url):
        print(f"Start downloading {url}")
        
        async with aiohttp.ClientSession() as s:
            resp = await s.get(url)
            #out = image_name_from_url(url), await resp.read()
            
        # print(resp[:55])
        # return out
        
    return await asyncio.gather(*[asyncio_request(url) for url in url_list])


In [17]:
def kiwi_api_call_async(dte_as_str, maxRows, rChunk):
    
    #cred_devtest_gcc = 'C:\gcpsdk\creds\sky-devandtest-gcc.json'
    cred_devtest_gcc = 'C:\\Users\\PaulDesmond\\OneDrive - SKYTRA LIMITED\\Automation\\gcp_keys\\'
    cred_devtest_gcc = cred_devtest_gcc + 'skytra-benchmark-devandtest-cloud-composer.json'
    client = bigquery.Client.from_service_account_json(cred_devtest_gcc)
    prj = 'skytra-benchmark-uat.'
    
    #client = bigquery.Client()
    #prj = ''
    
    qry = f"""select api_url from {prj}log.kiwi_api_url_hist where api_date = DATE'{dte_as_str}' 
        order by tick_count desc LIMIT {maxRows}"""
    query_job = client.query(qry, location = 'EU') 
    df = query_job.result().to_dataframe()
    
    rMax = len(df.index)
    i = 0
    
    # https://stackoverflow.com/questions/55409641/asyncio-run-cannot-be-called-from-a-running-event-loop#:~:text=run()%20documentation%20says%3A,running%20in%20the%20same%20thread.&text=That's%20the%20reason%20why%20you,call%20await%20main(url)%20.
    # *** asyncio.run() cannot be called from a running event loop ***************
    # I think this is more to do with Jupyter / if __name__ == "__main__"   rather than While Loop
    
    while i * rChunk < rMax:

        dfSubset = df[i * rChunk:i * rChunk + rChunk]

        # orig method tested but cannot see request result ??
        # async_request_chunk(dfSubset)

        url_list = dfSubset.api_url.tolist()
        # print(type(dfSubset.api_url))
        # print(type(url_list))

        st = time.time()

        # apparently in Jupyter syntax is different
        #if __name__ == "__main__":
            #reqResults = await asyncio_request_batch(url_list)
            #run_async(asyncio_request_batch, url_list)

        # non-Jupyter
        reqResults = asyncio.run(asyncio_request_batch(url_list))

        print(f"Asynchronous exec took {time.time() - st} seconds")

        i = i + 1


In [11]:
#cred_devtest_gcc = 'C:\\Users\\PaulDesmond\\OneDrive - SKYTRA LIMITED\\Automation\\gcp_keys\\'
#cred_devtest_gcc = cred_devtest_gcc + 'skytra-benchmark-devandtest-cloud-composer.json'
#cred_devtest_gcc

'C:\\Users\\PaulDesmond\\OneDrive - SKYTRA LIMITED\\Automation\\gcp_keys\\skytra-benchmark-devandtest-cloud-composer.json'

In [18]:
kiwi_api_call_async('2020-12-09', 50, 25)
print('finished')

c:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  from ipykernel import kernelapp as app


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
# orig DONT CHANGE
# https://towardsdatascience.com/stop-waiting-start-using-async-and-await-18fcd1c28fd0

"""
async def donwload_aio(urls:Iterable[str])->List[Tuple[str, bytes]]:
    async def download(url: str) -> Tuple[str, bytes]:
        print(f"Start downloading {url}")
        async with aiohttp.ClientSession() as s:
            resp = await s.get(url)
            out = image_name_from_url(url), await resp.read()
        print(f"Done downloading {url}")
        return out
        
    return await asyncio.gather(*[download(url) for url in urls])


# Take only 200 images to not run into issues with gather     
reduced = images[:200]
st = time.time()

images_a = asyncio.run(donwload_aio(reduced))
"""

In [ ]:
"""
from multiprocessing.dummy import Pool
import requests

def async_request_chunk(dfSubset):
    pool = Pool(len(dfSubset.index))

    reqSet = []
    for i in dfSubset.index:
        
        reqSet.append(pool.apply_async( requests.get, [dfSubset.api_url[i]] ))
        # print(dfSubset.api_url[i])
        
    for req in reqSet:
        try:
            r = req.get() # For each req, wait until the request is finished and then print the response object
            print(r[:55])
        except:
            pass
"""

In [ ]:
"""
#https://stackoverflow.com/questions/55409641/asyncio-run-cannot-be-called-from-a-running-event-loop#:~:text=run()%20documentation%20says%3A,running%20in%20the%20same%20thread.&text=That's%20the%20reason%20why%20you,call%20await%20main(url)%20.

import threading

class RunThread(threading.Thread):
    def __init__(self, func, args, kwargs):
        self.func = func
        self.args = args
        self.kwargs = kwargs
        super().__init__()

    def run(self):
        self.result = asyncio.run(self.func(*self.args, **self.kwargs))

def run_async(func, *args, **kwargs):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = None
    if loop and loop.is_running():
        thread = RunThread(func, args, kwargs)
        thread.start()
        thread.join()
        
        # return thread.result
        
    else:
        return asyncio.run(func(*args, **kwargs))
"""